In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.utils import resample
from sklearn.svm import SVC, LinearSVC
from imblearn.over_sampling import SMOTE
import seaborn as sns

In [2]:
data = pd.read_csv('./dataForTeammates/new_data.csv')

In [3]:
data.columns

Index(['Unnamed: 0', 'encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'target', 'self_pay',
       'medicaid/medicare', 'coverByInsurance', 'f_diag', 's_diag', 't_diag',
       'clustering3', 'clustering4', 'clustering2'],


In [4]:
data = data.drop(['Unnamed: 0','encounter_id','patient_nbr'],axis = 1)
data.columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'medical_specialty', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'target', 'self_pay',
       'medicaid/medicare', 'coverByInsurance', 'f_diag', 's_diag', 't_diag',
       'clustering3', 'clustering4', 'clustering2'],
      dtype='object')

In [5]:
data.shape

(86500, 52)

In [6]:
num_columns = list(data.select_dtypes("int64").columns)
num_columns.remove('admission_type_id')
num_columns.remove('discharge_disposition_id')
num_columns.remove('admission_source_id')
num_columns.remove('target')
num_columns.remove('self_pay')
num_columns.remove('medicaid/medicare')
num_columns.remove('coverByInsurance')
num_columns.remove('clustering3')
num_columns.remove('clustering4')
num_columns.remove('clustering2')

xnames = []
for i in data.columns:
    if i != 'target':
        xnames.append(i)
x = data[xnames]
y = data['target']
for i in num_columns:
    print(type(i))
    
num_columns

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


['time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'number_diagnoses']

In [7]:
data.dtypes #

race                        object
gender                      object
age                         object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
medical_specialty           object
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride                 object
acetohexamide               object
glipizide                   object
glyburide                   object
tolbutamide                 object
pioglitazone                object
rosiglitazone               object
acarbose            

In [8]:
data = data.drop(['glimepiride', 'examide', 'tolazamide', 'glyburide-metformin','glipizide-metformin', 'glimepiride-pioglitazone','metformin-rosiglitazone', 'troglitazone'], axis = 1)

In [9]:
y = data['target']

In [10]:
data = data.drop(['target'], axis = 1)

In [11]:
data['admission_type_id'] = data['admission_type_id'].apply(str)

In [12]:
data['discharge_disposition_id'] = data['discharge_disposition_id'].apply(str)

In [13]:
data['admission_source_id'] = data['admission_source_id'].apply(str)

In [14]:
dummied_features = pd.get_dummies(data, drop_first = True)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(dummied_features, y, test_size=0.3, random_state=42, stratify=y, shuffle = True)

In [16]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()

In [17]:
cols_num = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_emergency', 'number_inpatient',
           'number_diagnoses']

In [18]:
scaler.fit(X_train[cols_num])

C:\Users\Leo\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [19]:
X_train[cols_num] = scaler.transform(X_train[cols_num])

C:\Users\Leo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.
C:\Users\Leo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Leo\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [20]:
X_test[cols_num] = scaler.transform(X_test[cols_num])

C:\Users\Leo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.
C:\Users\Leo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Leo\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [21]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
x_train_under, y_train_under = rus.fit_sample(X_train, y_train)
#x_train_under.shape
x_train_under.shape

(13514, 243)

In [22]:
sm = SMOTE(random_state=42)

x_sm, y_sm = sm.fit_sample(X_train, y_train)

lsvc = LinearSVC(class_weight = "balanced")
svc = SVC(class_weight = "balanced")
x_sm.shape

(107586, 243)

In [29]:
import pickle
final_svc = open("svc_pickle.sav","rb")
svc = pickle.load(final_svc)

C:\Users\Leo\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator SVC from version 0.19.2 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [31]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, recall_score,roc_auc_score, precision_score, accuracy_score

In [32]:
print("auc train score: ", roc_auc_score(y_train_under, svc.predict(x_train_under)))
print("auc test score: ",roc_auc_score(y_test, svc.predict(x_test)))
print("recall train score: ",recall_score(y_train_under, svc.predict(x_train_under)))
print("recall test score: ",recall_score(y_test, svc.predict(x_test)))
print("precision train score: ", precision_score(y_train_under, svc.predict(x_train_under)))
print("precision test score: ",precision_score(y_test, svc.predict(x_test)))
print("accuracy train score: ",accuracy_score(y_train_under, svc.predict(x_train_under)))
print("accuracy test score: ",accuracy_score(y_test, svc.predict(x_test)))
print(confusion_matrix(y_test, svc.predict(x_test)))

ValueError: X.shape[1] = 243 should be equal to 280, the number of features at training time